In [11]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import zoom
import ipywidgets as widgets
from IPython.display import display
import os
from collections import defaultdict
from tqdm import tqdm  
import random
from tqdm.notebook import tqdm

Inital File Set Orgnaizaiton

In [ ]:
directories = ["D:\\CTH_archive\\TMAX_DICOM", "D:\\CTH_archive\\CTH_DICOM_RESIZE", "D:\\CTH_archive\\CTP_DICOM"]
directory_names = ["TMAX_DICOM", "CTH_DICOM_RESIZE", "CTP_DICOM"]

# Initialize a dictionary to store file paths organized by patient name/MRN
patient_files = defaultdict(lambda: defaultdict(list))

# Process each directory
for directory, dir_name in zip(directories, directory_names):
    print(f"Processing directory: {directory}")
    if os.path.exists(directory):
        for root, _, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                # Split the path to extract the patient name
                parts = root.split("\\")
                # Find the index of the main directory to ensure the patient name is correctly extracted
                try:
                    index = parts.index(dir_name) + 1
                    patient_name = parts[index]
                    patient_files[patient_name][dir_name].append(file_path)
                except (IndexError, ValueError) as e:
                    print(f"Error processing {file_path}: {e}")
    else:
        print(f"Error: Directory {directory} does not exist")

# Filter patients with files in all three directories and store in a new variable
filtered_patient_files = {
    patient: folders for patient, folders in patient_files.items()
    if set(folders.keys()) == set(directory_names)
}

print("Number of patients with all folders:", len(filtered_patient_files))

In [13]:
def read_dicom_series(directory):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    if image.GetNumberOfComponentsPerPixel() > 1:
        channels = [sitk.VectorIndexSelectionCast(image, i) for i in range(image.GetNumberOfComponentsPerPixel())]
        image = sum(channels) / len(channels)

    image = sitk.Cast(image, sitk.sitkFloat32)
    return image


In [14]:
transforms_dir = r'D:\CTH_archive\Transforms'  # Directory to save the transforms
if not os.path.exists(transforms_dir):
    os.makedirs(transforms_dir)

resampled_images = {}  # To store the resampled images for each patient
resampled_images_transform = {}  # To store the transform files for each patient

def remove_background(image):
    threshold_value = -300  # Adjust based on your CT scans
    binary_image = sitk.BinaryThreshold(image, lowerThreshold=threshold_value, upperThreshold=3000, insideValue=1, outsideValue=0)
    morph_radius = [2, 2, 2]  # Specify as a list or tuple
    binary_image = sitk.BinaryMorphologicalOpening(binary_image, morph_radius)
    cc_filter = sitk.ConnectedComponentImageFilter()
    cc_image = cc_filter.Execute(binary_image)
    stats_filter = sitk.LabelIntensityStatisticsImageFilter()
    stats_filter.Execute(cc_image, binary_image)
    largest_label = max(stats_filter.GetLabels(), key=lambda x: stats_filter.GetPhysicalSize(x))
    binary_image = sitk.BinaryThreshold(cc_image, lowerThreshold=largest_label, upperThreshold=largest_label, insideValue=1, outsideValue=0)
    
    # Cast the binary_image to the same pixel type as the original image
    casted_binary_image = sitk.Cast(binary_image, image.GetPixelID())

    return image * casted_binary_image

for patient in tqdm(patients_with_all_folders, desc='Processing Patients'):
    if 'CTP_DICOM' in patient_files[patient] and 'CTH_DICOM_RESIZE' in patient_files[patient]:
        ctp_dicom_directory = os.path.dirname(patient_files[patient]['CTP_DICOM'][0])
        CTH_DICOM_RESIZE_directory = os.path.dirname(patient_files[patient]['CTH_DICOM_RESIZE'][0])

        # Read DICOM series
        moving_image = read_dicom_series(ctp_dicom_directory)
        fixed_image = read_dicom_series(CTH_DICOM_RESIZE_directory)
        
        # Resize the fixed image to 512x512
        #fixed_image = resize_image(fixed_image)

        # Remove background from the moving and fixed images
        segmented_moving_image = remove_background(moving_image)
        segmented_fixed_image = remove_background(fixed_image)
        
    
           # Read DICOM series and apply histogram normalization
        moving_image = read_dicom_series(ctp_dicom_directory)
        moving_image_min = float(sitk.GetArrayFromImage(moving_image).min())
        moving_image_max = float(sitk.GetArrayFromImage(moving_image).max())
        moving_image = sitk.IntensityWindowing(moving_image,
                                            windowMinimum=moving_image_min,
                                            windowMaximum=moving_image_max,
                                            outputMinimum=0.0,
                                            outputMaximum=1.0)

        fixed_image = read_dicom_series(CTH_DICOM_RESIZE_directory)
        fixed_image_min = float(sitk.GetArrayFromImage(fixed_image).min())
        fixed_image_max = float(sitk.GetArrayFromImage(fixed_image).max())
        fixed_image = sitk.IntensityWindowing(fixed_image,
                                            windowMinimum=fixed_image_min,
                                            windowMaximum=fixed_image_max,
                                            outputMinimum=0.0,
                                            outputMaximum=1.0)


        #fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        #axs[0, 0].imshow(sitk.GetArrayFromImage(fixed_image)[fixed_image.GetSize()[2] // 2], cmap='gray')
        #axs[0, 0].set_title('Original Fixed Image')
        #axs[0, 0].axis('off')
        #axs[0, 1].imshow(sitk.GetArrayFromImage(segmented_fixed_image)[fixed_image.GetSize()[2] // 2], cmap='gray')
        #axs[0, 1].set_title('Segmented Fixed Image')
        #axs[0, 1].axis('off')
        #axs[1, 0].imshow(sitk.GetArrayFromImage(moving_image)[moving_image.GetSize()[2] // 2], cmap='gray')
        #axs[1, 0].set_title('Original Moving Image')
        #axs[1, 0].axis('off')
        #axs[1, 1].imshow(sitk.GetArrayFromImage(segmented_moving_image)[moving_image.GetSize()[2] // 2], cmap='gray')
        #axs[1, 1].set_title('Segmented Moving Image')
        #axs[1, 1].axis('off')
        #plt.show()

        registration_method = sitk.ImageRegistrationMethod()
        registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
        registration_method.SetMetricSamplingPercentage(0.6, sitk.sitkWallClock)
        registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
        registration_method.SetOptimizerAsGradientDescentLineSearch(learningRate=0.5, numberOfIterations=300, convergenceMinimumValue=1e-6, convergenceWindowSize=20)
        registration_method.SetOptimizerScalesFromPhysicalShift()
        registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[8, 4, 2])
        registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[4, 2, 1])
        registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
        initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.AffineTransform(fixed_image.GetDimension()), sitk.CenteredTransformInitializerFilter.MOMENTS)
        registration_method.SetInitialTransform(initial_transform)
        try:
            final_transform = registration_method.Execute(segmented_fixed_image, segmented_moving_image)
            resampled_image = sitk.Resample(segmented_moving_image, segmented_fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
            resampled_images[patient] = resampled_image
            transform_file = os.path.join(transforms_dir, f'{patient}_transform.h5')
            sitk.WriteTransform(final_transform, transform_file)
            resampled_images_transform[patient] = transform_file
            print(f"Registration successful for patient: {patient}. Transform saved to {transform_file}")
        except RuntimeError as e:
            print(f"Registration failed for patient {patient}: {e}")



Processing Patients:   0%|          | 0/85 [00:00<?, ?it/s]

Registration successful for patient: ALFORD_BARBARA 4024996. Transform saved to D:\CTH_archive\Transforms\ALFORD_BARBARA 4024996_transform.h5
Registration successful for patient: ALLAH_MAJUSTICE 2621774. Transform saved to D:\CTH_archive\Transforms\ALLAH_MAJUSTICE 2621774_transform.h5
Registration successful for patient: BATTLE_MARIA 8399298. Transform saved to D:\CTH_archive\Transforms\BATTLE_MARIA 8399298_transform.h5
Registration successful for patient: BOGER_DAVID_S 2532249. Transform saved to D:\CTH_archive\Transforms\BOGER_DAVID_S 2532249_transform.h5
Registration successful for patient: BROWN_ANTHONY F341525. Transform saved to D:\CTH_archive\Transforms\BROWN_ANTHONY F341525_transform.h5
Registration successful for patient: CAMPAGNA_HARRY_D E304749. Transform saved to D:\CTH_archive\Transforms\CAMPAGNA_HARRY_D E304749_transform.h5
Registration successful for patient: CANIGLIA_ROBERT 2363078. Transform saved to D:\CTH_archive\Transforms\CANIGLIA_ROBERT 2363078_transform.h5
Regist

In [15]:
def display_image_slices(fixed_image, moving_image, transformed_image):
    # Convert SimpleITK images to arrays for easier manipulation
    fixed_image_array = sitk.GetArrayFromImage(fixed_image)
    transformed_image_array = sitk.GetArrayFromImage(transformed_image)

    # Determine the maximum number of slices from the fixed and transformed images
    max_slices = max(fixed_image.GetSize()[2], transformed_image.GetSize()[2])

    def update_slice(slice_idx):
        # Create figure with 2 subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        # Display the fixed image slice
        axs[0].imshow(fixed_image_array[slice_idx], cmap='gray')
        axs[0].set_title('Fixed Image')

        # Display the overlay: fixed image slice with the transformed image slice overlaid with transparency
        if slice_idx < transformed_image_array.shape[0]:
            axs[1].imshow(fixed_image_array[slice_idx], cmap='gray')
            axs[1].imshow(transformed_image_array[slice_idx], cmap='jet', alpha=0.5)  # Adjust alpha for desired transparency
            axs[1].set_title('Overlay: Fixed + Transformed Image')
        else:
            axs[1].text(0.5, 0.5, 'Slice not available', horizontalalignment='center', verticalalignment='center')
            axs[1].set_title('Overlay: Fixed + Transformed Image')

        plt.show()

    # Create a slider widget for slice selection
    slice_slider = widgets.IntSlider(min=0, max=max_slices-1, step=1, value=max_slices//2, description='Slice')

    # Display the widget and use `interactive_output` to connect the slider with the update function
    interactive_output = widgets.interactive_output(update_slice, {'slice_idx': slice_slider})
    display(slice_slider, interactive_output)

In [16]:
def display_fusion(fixed_image, registered_image):
    fixed_array = sitk.GetArrayFromImage(fixed_image)
    registered_array = sitk.GetArrayFromImage(registered_image)

    # Ensure the images are in the same size for overlay
    registered_resampled = sitk.Resample(registered_image, fixed_image)

    # Convert SimpleITK images to arrays
    fixed_array = sitk.GetArrayFromImage(fixed_image)
    registered_array = sitk.GetArrayFromImage(registered_resampled)

    # Choose a slice in the middle of the volume to display
    slice_idx = fixed_array.shape[0] // 2

    # Create a fusion image by overlaying the registered image on the fixed image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(fixed_array[slice_idx], cmap='gray')
    plt.title('Fixed Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(fixed_array[slice_idx], cmap='gray')
    plt.imshow(registered_array[slice_idx], cmap='jet', alpha=0.5)  # Adjust alpha for transparency
    plt.title('Fusion Visualization')
    plt.axis('off')

    plt.show()

In [ ]:
#for patient_id in patient_files.keys():
#    # Check if 'CTP_DICOM' and 'CTH_DICOM_RESIZE' lists have elements
#    if patient_files[patient_id]['CTP_DICOM'] and patient_files[patient_id]['CTH_DICOM_RESIZE']:
#        # Define directories
#        ctp_dicom_directory = os.path.dirname(patient_files[patient_id]['CTP_DICOM'][0])
#        CTH_DICOM_RESIZE_directory = os.path.dirname(patient_files[patient_id]['CTH_DICOM_RESIZE'][0])
#
#        # Read DICOM series
#        moving_image = read_dicom_series(ctp_dicom_directory)
#        fixed_image = read_dicom_series(CTH_DICOM_RESIZE_directory)
#
#        # Assuming transformations have been applied and the transformed images are stored
#        transformed_image = resampled_images.get(patient_id)  # Using .get() to avoid KeyError if patient_id is not in resampled_images
#
#        if transformed_image:  # Check if transformed_image exists for the patient_id
#            # Display fusion for the middle slice of each patient's fixed and transformed images
#            display_fusion(fixed_image, transformed_image)
#            pass
#        else:
#            print(f"No transformed image for patient {patient_id}")
#    else:
#        print(f"Missing DICOM files for patient {patient_id}")
#

In [ ]:
def convert_series_to_nifti(input_directory, output_file):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(input_directory)
    reader.SetFileNames(dicom_names)
    image_series = reader.Execute()
    
    # Convert to numpy array to manipulate the pixel data directly
    img_array = sitk.GetArrayFromImage(image_series)

    # Check if the image needs to be converted to grayscale
    if image_series.GetNumberOfComponentsPerPixel() > 1:
        img_array = np.mean(img_array, axis=-1).astype(np.uint16)

        img_array[:, :35, :] = 0  # Remove TAMX label
        img_array[:, :, :35] = 0 # Remove scale on  the left
    
    # Convert the numpy array back to a SimpleITK Image
    processed_image = sitk.GetImageFromArray(img_array)
    processed_image.SetSpacing(image_series.GetSpacing())
    processed_image.SetOrigin(image_series.GetOrigin())
    processed_image.SetDirection(image_series.GetDirection())

    # Write the processed image as a NIfTI file
    sitk.WriteImage(processed_image, output_file)

root_directory = r'D:\CTH_archive\TMAX_DICOM'

for patient_dir in os.listdir(root_directory):
    patient_path = os.path.join(root_directory, patient_dir)
    if os.path.isdir(patient_path):
        for series_dir in os.listdir(patient_path):
            series_path = os.path.join(patient_path, series_dir)
            if os.path.isdir(series_path):
                output_nifti_file = os.path.join(patient_path, f"{series_dir}.nii")
                print(f"Converting {series_path} to NIfTI...")
                convert_series_to_nifti(series_path, output_nifti_file)

In [21]:
CTH_ARCHIVE_DIR = r'D:\CTH_archive'
TMAX_DICOM_DIR = os.path.join(CTH_ARCHIVE_DIR, 'TMAX_DICOM')
CTP_DICOM_DIR = os.path.join(CTH_ARCHIVE_DIR, 'CTP_DICOM')
CTH_DICOM_RESIZE_DIR = os.path.join(CTH_ARCHIVE_DIR, 'CTH_DICOM_RESIZE')
REGISTERED_NIFTI_DIR = os.path.join(CTH_ARCHIVE_DIR, 'Registered_NIfTI')

if not os.path.exists(REGISTERED_NIFTI_DIR):
    os.makedirs(REGISTERED_NIFTI_DIR)

# Function to resample the moving image
def resample_image(moving_image, fixed_image, ctp_image):
    desired_size = [fixed_image.GetSize()[0], fixed_image.GetSize()[1], ctp_image.GetSize()[2]]
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(moving_image)
    resampler.SetSize(desired_size)
    resampler.SetOutputSpacing([moving_image.GetSpacing()[i] * (moving_image.GetSize()[i] / desired_size[i]) for i in range(3)])
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkLinear)
    resized_moving_image = resampler.Execute(moving_image)
    resized_moving_image.SetSpacing(ctp_image.GetSpacing())
    resized_moving_image.SetOrigin(ctp_image.GetOrigin())
    resized_moving_image.SetDirection(ctp_image.GetDirection())
    return resized_moving_image

# Function to apply the final transform to the resized moving image
def apply_final_transform(resized_moving_image, fixed_image, transform_file_path):
    if not os.path.exists(transform_file_path):
        print(f"Transform file not found at {transform_file_path}.")
        return None
    final_transform = sitk.ReadTransform(transform_file_path)
    resampled_image = sitk.Resample(resized_moving_image, 
                                    fixed_image, 
                                    final_transform, 
                                    sitk.sitkLinear, 
                                    0.0, 
                                    fixed_image.GetPixelID())
    return resampled_image

# Function to save the resampled image
def save_resampled_image(resampled_image, patient_id):
    output_path = os.path.join(REGISTERED_NIFTI_DIR, f'{patient_id}_registered.nii')
    sitk.WriteImage(resampled_image, output_path)
    print(f'output_path: {output_path}')

# Function to process each patient
def process_patient(patient_id):
    patient_directory = os.path.join(TMAX_DICOM_DIR, patient_id)
    nifti_files = [f for f in os.listdir(patient_directory) if f.endswith('.nii')]
    if not nifti_files:
        print(f"No NIfTI file found for patient {patient_id}.")
        return None, None, None
    nifti_file = nifti_files[0]
    nifti_file_path = os.path.join(patient_directory, nifti_file)
    moving_image = sitk.ReadImage(nifti_file_path)
    ctp_directory_base = os.path.join(CTP_DICOM_DIR, patient_id)
    study_dirs = [d for d in os.listdir(ctp_directory_base) if os.path.isdir(os.path.join(ctp_directory_base, d))]
    if not study_dirs:
        print(f"No CTP study found for patient {patient_id} in {ctp_directory_base}.")
        return None, None, None
    ctp_directory = os.path.join(ctp_directory_base, study_dirs[0])
    ctp_image = read_dicom_series(ctp_directory)
    fixed_image_base_dir = os.path.join(CTH_DICOM_RESIZE_DIR, patient_id)
    series_dirs = [d for d in os.listdir(fixed_image_base_dir) if os.path.isdir(os.path.join(fixed_image_base_dir, d))]
    if not series_dirs:
        print(f"No DICOM series found for patient {patient_id} in {fixed_image_base_dir}.")
        return None, None, None
    fixed_image_dir = os.path.join(fixed_image_base_dir, series_dirs[0])
    fixed_image = read_dicom_series(fixed_image_dir)
    resized_moving_image = resample_image(moving_image, fixed_image, ctp_image)
    transform_file_path = resampled_images_transform[patient_id]
    resampled_image = apply_final_transform(resized_moving_image, fixed_image, transform_file_path)
    if resampled_image is not None:
        save_resampled_image(resampled_image, patient_id)
        print(f"Processed and saved registered image for patient: {patient_id}")
    return fixed_image, moving_image, resampled_image

# Main function to process all patients
def process_all_patients():
    successful_patients = list(resampled_images.keys())
    random_subset = random.sample(successful_patients, 20)  # Select 20 random patient IDs
    for patient_id in successful_patients:
        fixed_image, moving_image, resampled_image = process_patient(patient_id)
        if patient_id in random_subset:
            display_image_slices(fixed_image, moving_image, resampled_image)
            print(f"Displayed image slices for patient: {patient_id}")

process_all_patients()

output_path: D:\CTH_archive\Registered_NIfTI\ALFORD_BARBARA 4024996_registered.nii
Processed and saved registered image for patient: ALFORD_BARBARA 4024996
output_path: D:\CTH_archive\Registered_NIfTI\ALLAH_MAJUSTICE 2621774_registered.nii
Processed and saved registered image for patient: ALLAH_MAJUSTICE 2621774
output_path: D:\CTH_archive\Registered_NIfTI\BATTLE_MARIA 8399298_registered.nii
Processed and saved registered image for patient: BATTLE_MARIA 8399298
output_path: D:\CTH_archive\Registered_NIfTI\BOGER_DAVID_S 2532249_registered.nii
Processed and saved registered image for patient: BOGER_DAVID_S 2532249
output_path: D:\CTH_archive\Registered_NIfTI\BROWN_ANTHONY F341525_registered.nii
Processed and saved registered image for patient: BROWN_ANTHONY F341525


IntSlider(value=16, description='Slice', max=31)

Output()

Displayed image slices for patient: BROWN_ANTHONY F341525
output_path: D:\CTH_archive\Registered_NIfTI\CAMPAGNA_HARRY_D E304749_registered.nii
Processed and saved registered image for patient: CAMPAGNA_HARRY_D E304749
output_path: D:\CTH_archive\Registered_NIfTI\CANIGLIA_ROBERT 2363078_registered.nii
Processed and saved registered image for patient: CANIGLIA_ROBERT 2363078
output_path: D:\CTH_archive\Registered_NIfTI\CARDIN_PAUL E570140_registered.nii
Processed and saved registered image for patient: CARDIN_PAUL E570140
output_path: D:\CTH_archive\Registered_NIfTI\CHANG_WAH_KONG 5708968_registered.nii
Processed and saved registered image for patient: CHANG_WAH_KONG 5708968


IntSlider(value=16, description='Slice', max=31)

...

Output()

Displayed image slices for patient: RYAN_MICHAEL 2967285
output_path: D:\CTH_archive\Registered_NIfTI\SALTER_GLADYS 3281623_registered.nii
Processed and saved registered image for patient: SALTER_GLADYS 3281623
output_path: D:\CTH_archive\Registered_NIfTI\SANTIAGO_CARMEN 7133486_registered.nii
Processed and saved registered image for patient: SANTIAGO_CARMEN 7133486
output_path: D:\CTH_archive\Registered_NIfTI\SCOUFARAS_DESPINA 3712417_registered.nii
Processed and saved registered image for patient: SCOUFARAS_DESPINA 3712417
output_path: D:\CTH_archive\Registered_NIfTI\SGARLATO_SANTO A244280_registered.nii
Processed and saved registered image for patient: SGARLATO_SANTO A244280
output_path: D:\CTH_archive\Registered_NIfTI\SHERPA_DEKI 5678752_registered.nii
Processed and saved registered image for patient: SHERPA_DEKI 5678752


IntSlider(value=14, description='Slice', max=27)

Output()

Displayed image slices for patient: SHERPA_DEKI 5678752
output_path: D:\CTH_archive\Registered_NIfTI\SIEJACK_EURANIA 2466204_registered.nii
Processed and saved registered image for patient: SIEJACK_EURANIA 2466204
output_path: D:\CTH_archive\Registered_NIfTI\SINGH_HARBHAJAN 9096349_registered.nii
Processed and saved registered image for patient: SINGH_HARBHAJAN 9096349
output_path: D:\CTH_archive\Registered_NIfTI\SINGLETARY_DONALD 3902533_registered.nii
Processed and saved registered image for patient: SINGLETARY_DONALD 3902533
output_path: D:\CTH_archive\Registered_NIfTI\SUAREZ_EDWIN E837730_registered.nii
Processed and saved registered image for patient: SUAREZ_EDWIN E837730
output_path: D:\CTH_archive\Registered_NIfTI\TORII_YUKIO E574468_registered.nii
Processed and saved registered image for patient: TORII_YUKIO E574468
output_path: D:\CTH_archive\Registered_NIfTI\TOUATI_MOHAMED 3117292_registered.nii
Processed and saved registered image for patient: TOUATI_MOHAMED 3117292
output_p

In [ ]:
#Resize and pad to 512x512 This does casue some distortion of the anatomy, there might be a better way

def resize_and_pad(array, target_size=(512, 512), order=1):
    """Resize a 2D array to the target size and pad with zeros to match the target size exactly."""
    zoom_factors = np.array(target_size) / np.array(array.shape)
    resized_array = zoom(array, zoom_factors, order=order)
    pad_width = [(0, 0)] * array.ndim  # Initialize pad width for each dimension
    for i, dim in enumerate(resized_array.shape):
        pad = (target_size[i] - dim) / 2
        pad_width[i] = (int(np.floor(pad)), int(np.ceil(pad)))
    padded_array = np.pad(resized_array, pad_width, mode='constant', constant_values=0)
    return padded_array.astype(array.dtype)

def process_series(series_path, dest_root):
    """Process a DICOM series by resizing and padding all slices, then save them in the new folder structure."""
    for file in os.listdir(series_path):
        if file.endswith(".dcm"):
            dicom_path = os.path.join(series_path, file)
            ds = pydicom.dcmread(dicom_path)

            if ds.pixel_array.ndim != 2:
                print(f"Skipping {dicom_path}: Expected a 2D array, got {ds.pixel_array.ndim}D array instead.")
                continue

            resized_padded_img = resize_and_pad(ds.pixel_array)
            ds.PixelData = resized_padded_img.tobytes()
            ds.Rows, ds.Columns = resized_padded_img.shape

            # Extract components from the source path
            components = series_path.split(os.sep)[-2:]  # Last two components should be Patient Name MRN and Date Time
            patient_info = components[0]
            date_time = components[1].replace(" ", "_")  # Replace spaces with underscores for consistency

            # Construct the destination path
            dest_path = os.path.join(dest_root, patient_info, date_time)

            if not os.path.exists(dest_path):
                os.makedirs(dest_path)

            ds.save_as(os.path.join(dest_path, file))

def process_base_folder(base_folder, dest_root):
    """Process all series in the base folder."""
    for root, dirs, files in os.walk(base_folder):
        if any(file.endswith('.dcm') for file in files):
            process_series(root, dest_root)

base_folder = "D:/CTH_archive/CTH_DICOM"
dest_root = "D:/CTH_archive/CTH_DICOM_RESIZE_RESIZE"
#process_base_folder(base_folder, dest_root)
